In [11]:
import cv2  #gerekli olan kütüphanelerimizi (paketleri) tanımlıyoruz
import pickle 
import numpy as np 
import random
from tensorflow.keras.preprocessing.image import img_to_array

image = cv2.imread('src/mnist.jpg')  #resmimizi path'i verilen yoldan içe aktarıyoruz

#ilklendir selective search
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)  #algoritmanın işlem yapacağı resmi aktarıyoruz
ss.switchToSelectiveSearchQuality()  #algoritmamızı içe aktardık 

print('start')
rects = ss.process()  #burda algoritma ile nesne tespiti ==> x ,y ,w ,h değerlerini döndürür

proposals = [] #roi ve koordinat değerleri için 2 adet boş bir liste tanımlıyoruz
boxes = []
output = image.copy() #orijinal resmimizin kopyasını oluşturuyoruz

for (x,y,w,h) in rects[:80]: #tespit edilmiş 80 adet nesnenin içinde geziniyoruz
    color = [random.randint(0,255) for j in range(0,3)] #random bir renk oluşturuyoruz
    cv2.rectangle(output,(x,y),(x+w,y+h),color,2)  #bulduğumuz koordinatlara rastgele renk ile dikdörtgen çizdiriyoruz

    roi = image[y:y+h,x:x+w]  #region of interest oluşturuyoruz
    roi = cv2.resize(roi,(32,32),interpolation = cv2.INTER_LANCZOS4)  #roi mizi 32x32 olmak üzere yeniden boyutlandırıyoruz
    roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY) #bgr formatından gray formatına çeviriyoruz
    
    roi = img_to_array(roi) #img formatından array formatına çeviriyoruz (model için)

    proposals.append(roi) #arrayleri proposals listesine ekliyoruz
    boxes.append((x,y,x+w,y+h)) #bu arrayların koordinatarını ise boxes listesine ekliyoruz

proposals = np.array(proposals,dtype='float64')  #bu listedeki elemanları numpy array formatına getiriyoruz
boxes = np.array(boxes,dtype='int32') #bu listedeki elemanları integer olmak şartıyla array formatına çeviriyoruz


pickle_in = open('model_trained.p','rb') #modeli içe aktarıyoruz
model = pickle.load(pickle_in) 
proba = model.predict(proposals)  #tahminleri döndürüyoruz

number_list = [] #sınıflandır için 2 farklı liste oluşturuyoruz
idx = []

#SINIFLANDIRMA
for i in range(len(proba)):  #tahminlerin içinde dolaşıyoruz
    max_prob = np.max(proba[i,:]) #i indeksine karşılık gelen satırının maksimum olasılığını max_prob değişkenine eşitlerzi

    if max_prob >0.95: #eğer ki olasılık 0.95'ten buyukse 
        idx.append(i) # o anki i değerini idx listesine 
        number_list.append(np.argmax(proba[i]))  #maks sayısında index numarasını number_list listesine ekleriz

for i in range(len(number_list)):
    j = idx[i]
    cv2.rectangle(image,(boxes[j,0],boxes[j,1]),(boxes[j,2],boxes[j,3]),(0,0,255),2)
    cv2.putText(image,str(np.argmax(proba[j])),(boxes[j,0] + 5,boxes[j,1] + 5),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)

    cv2.imshow('img',image)
    

cv2.waitKey(0)
cv2.destroyAllWindows()

start
3/3 [==============================] - 0s 5ms/step
